In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime, date, timedelta
import yfinance as yfin
yfin.pdr_override()


# Graph
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
#import seaborn as sns


# Models
#import arviz as az
#import pymc as pm
import pymc3 as pm
#import xarray as xr
#import bambi as bmb
#from pymc import HalfCauchy, Model, Normal, sample

import warnings

In [2]:
pd.set_option('display.float_format', lambda x: f'{x:,.6f}')
plt.rcParams["figure.figsize"] = (10,6)

#%config InlineBackend.figure_format = 'retina'
#az.style.use("arviz-darkgrid")

#sk.set_config(display='diagram')
warnings.filterwarnings("ignore")

# Assignment

## Variables

In [3]:
start_date = datetime(2022, 1, 1)
purchase_date = datetime(2023, 9, 11)
split_date = datetime(2023,8,24)
today = datetime(2023,10,20)

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

#start = datetime.datetime(2022, 10, 1)
#purchase_date_str = '2023-9-11'
#end_date = datetime.now()
#end_date_str = end_date.strftime("%Y-%m-%d")

## Functions

# Load Data

In [4]:
folio_df = pd.read_csv('https://raw.githubusercontent.com/dsimband/DATA618/main/w12/data/DATA618_Portfolio_Rebalance_v2.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })

folio_df = folio_df[folio_df['Shares'] > 0]
folio_df = folio_df.groupby(['Ticker','BondName','Class','Sub_Class'])[['Shares','Value']].sum()
folio_df.reset_index(inplace=True)

# calculate portfilio percentage
portfolio_total = folio_df['Value'].sum()
folio_df['port_percent'] = folio_df['Value'] / portfolio_total

# Class Portfolios
folio_econ_df = folio_df[folio_df['Class'] == 'Economically Sensitive']
folio_int_df = folio_df[folio_df['Class'] == 'Interest Rate Sensitive']

print('folio_df: ',folio_df.shape)

folio_df:  (40, 7)


In [5]:
# ticker symbols
ticker_lst = list(folio_df['Ticker'])
print('ticker #:', len(ticker_lst))

# portfolio weights
weight_lst = (folio_df['port_percent'].values)
print('price #:', len(weight_lst))

#Download closing prices
price_df = pdr.get_data_yahoo(ticker_lst, start=purchase_date, end=today)['Close']
price_df['C_A_S_H'] = 1
print('price_df #:', price_df.shape)

#From the closing prices, calculate periodic returns
return_df = price_df.pct_change()
return_df.fillna(0, inplace=True)
return_df.index = pd.to_datetime(return_df.index)
print('return_df #:', len(return_df.columns))

ticker #: 40
price #: 40
[*********************100%%**********************]  40 of 40 completed


1 Failed download:
['C_A_S_H']: Exception('%ticker%: No timezone found, symbol may be delisted')



price_df #: (29, 40)
return_df #: 40


twlo_df = return_df[['TWLO']]
twlo_df.reset_index(inplace=True)
twlo_df.columns =['x','y']
twlo_df

In [95]:
#returns = return_df[['IVV']].T.values
returns = return_df.T.values
returns

array([[ 0.        , -0.00106375,  0.        , ..., -0.00655732,
        -0.0033003 , -0.00441501],
       [ 0.        , -0.0132287 , -0.01317878, ...,  0.01687001,
        -0.04466565, -0.02190755],
       [ 0.        , -0.00369206,  0.00171031, ...,  0.00145273,
        -0.01218457, -0.00293681],
       ...,
       [ 0.        , -0.00193209, -0.00539276, ...,  0.00482727,
        -0.01821728, -0.01356525],
       [ 0.        , -0.01173707,  0.00363284, ..., -0.00282242,
        -0.01485993, -0.00818847],
       [ 0.        , -0.00081336,  0.00061048, ..., -0.00458814,
        -0.00272368, -0.00462179]])

In [96]:
weight_lst

array([0.01156966, 0.01156966, 0.00578483, 0.00867724, 0.04839587,
       0.02603172, 0.00578483, 0.03239503, 0.04627862, 0.00578483,
       0.00578483, 0.01156966, 0.0636331 , 0.02313931, 0.04627862,
       0.03470897, 0.04627862, 0.04627862, 0.02892414, 0.01156966,
       0.01156966, 0.01156966, 0.00578483, 0.00578483, 0.04859255,
       0.01156966, 0.01156966, 0.03470897, 0.01156966, 0.06941793,
       0.00578483, 0.01156966, 0.0636331 , 0.01156966, 0.02313931,
       0.03470897, 0.03760138, 0.02313931, 0.03470897, 0.01156966])

In [97]:
#return_mean = np.mean(return_df[['IVV']])
return_mean = np.mean(return_df)
return_mean

-0.002384513919992051

In [98]:
#return_sigma = return_df[['IVV']].std().values[0]
return_sigma = return_df.std().values[0]
return_sigma

0.0031223798253673797

return_mean = return_df[['IVV','TWLO']]
#t0_df.reset_index(inplace=True)
return_mean = pd.DataFrame(return_mean.mean(axis=0))
return_mean

#t0_df[t0_df['index'] == 'IVV']

return_mean = np.mean(return_df['IVV'])
return_mean

return_sigma = return_df[['IVV','TWLO']].std()
return_sigma = pd.DataFrame(return_sigma)
return_sigma

t0_df = pd.DataFrame()
t0_df['mean'] = return_mean
t0_df['sigma'] = return_sigma
#t0_df['mu'] = None

with pm.Model() as portfolio_model:
    for i, row in t0_df.iterrows():
        print(i)
        #row['mu'] = pm.Normal('mu', mu=row['mean'], sigma=row['sigma'])


t0_df

with pm.Model() as portfolio_model:
    #mu = pm.Normal('mu', mu=-0.0017, sigma=0.0082)
    mu = pm.Normal('mu', mu=return_mean, sigma=return_sigma)

#mu = pm.Normal('mu', mu=-0.0017, sigma=0.0082)
mu = pm.Normal('mu', mu=return_mean.values, sigma=return_sigma.values)

t = pdr.get_data_yahoo('AAPL', start=purchase_date, end=today)
t.columns

num_assets = len(weight_lst)
num_assets

z = np.array(weight_lst)
print(z)

#list(return_df.columns)

# Model 

In [102]:
num_assets = len(weight_lst)

# Define the model for portfolio volatility
with pm.Model() as portfolio_model:
    # Prior distributions for asset weights
    #w = pm.Dirichlet('weights', a=np.ones(num_assets))
    w = pm.Dirichlet('weights', a=np.ones(num_assets))
    
    # Expected portfolio return (can be adjusted based on your assumptions)
    #mu = pm.Normal('mu', mu=0, sigma=0.01)
    #portfolio_return = pm.Deterministic('portfolio_return', pm.math.dot(w, [mu] * num_assets))
    
    mu = pm.Normal('mu', mu=return_mean, sigma=return_sigma)
    #mu = pm.Normal('mu', mu=-0.0017, sigma=0.0082)
    portfolio_return = pm.Deterministic('portfolio_return', pm.math.dot(w, [mu] * num_assets))
    
    # Covariance matrix (prior distribution)
    cov_matrix = pm.Lognormal('cov_matrix', mu=np.log(0.01), sigma=0.5, shape=(num_assets, num_assets))
    
    # Portfolio volatility (standard deviation)
    portfolio_volatility = pm.Deterministic('portfolio_volatility', pm.math.sqrt(pm.math.dot(w, pm.math.dot(cov_matrix, w.T))))
    
    # Likelihood (observed data)
    returns_observed = pm.Normal('returns_observed', mu=portfolio_return, sigma=portfolio_volatility, observed=returns)
    
    # Sampling
    trace = pm.sample(2000, tune=1000)
    
    # Forecasting 10 days of volatility using posterior predictive sampling
    future_returns = np.random.normal(0.001, 0.02, size=(10, 1))  # Simulated future returns
    future_data = {returns_observed: future_returns}
    
    # Performing posterior predictive sampling for future volatility
    posterior_pred = pm.sample_posterior_predictive(trace, var_names=['portfolio_volatility'], samples=10, model=portfolio_model)
    
    # Extracting and printing forecasted volatility for the next 10 days
    future_volatility = posterior_pred['portfolio_volatility'][-1]
    print("Forecasted volatility for the next 10 days:")
    print(future_volatility)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [cov_matrix, mu, weights]


/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/dsimbandumwe/anaconda3/envs/pymc3/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 21 seconds.
There were 856 divergences after tuning. Increase `target_accept` or reparameterize.
There were 1075 divergences after t

Forecasted volatility for the next 10 days:
0.013026808319238779


In [100]:
# Summary statistics of posterior distributions
summary = pm.summary(trace)
print(summary)

Got error No model on context stack. trying to find log_likelihood in translation.


                          mean       sd    hdi_3%   hdi_97%  mcse_mean  \
mu                   -0.002000 0.000000 -0.003000 -0.002000   0.000000   
weights[0]            0.000000 0.000000  0.000000  0.000000   0.000000   
weights[1]            0.000000 0.000000  0.000000  0.000000   0.000000   
weights[2]            0.000000 0.000000  0.000000  0.000000   0.000000   
weights[3]            0.000000 0.000000  0.000000  0.000000   0.000000   
...                        ...      ...       ...       ...        ...   
cov_matrix[39, 36]    0.011000 0.006000  0.003000  0.021000   0.000000   
cov_matrix[39, 37]    0.011000 0.006000  0.003000  0.022000   0.000000   
cov_matrix[39, 38]    0.011000 0.006000  0.003000  0.021000   0.000000   
cov_matrix[39, 39]    0.009000 0.007000  0.000000  0.020000   0.002000   
portfolio_volatility  0.013000 0.000000  0.012000  0.013000   0.000000   

                      mcse_sd     ess_bulk     ess_tail    r_hat  
mu                   0.000000 2,559.000000 2

In [13]:
summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,-0.002000,0.004000,-0.009000,0.005000,0.000000,0.000000,"5,498.000000","4,205.000000",1.000000
weights[0],1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,"8,000.000000","8,000.000000",NaN
portfolio_return,-0.002000,0.004000,-0.009000,0.005000,0.000000,0.000000,"5,498.000000","4,205.000000",1.000000
"cov_matrix[0, 0]",0.001000,0.000000,0.000000,0.001000,0.000000,0.000000,"5,364.000000","5,048.000000",1.000000
portfolio_volatility,0.023000,0.005000,0.014000,0.032000,0.000000,0.000000,"5,364.000000","5,048.000000",1.000000


0.013065585735920007